In [23]:
# !git clone https://github.com/shireenchand/Crework-Sign-Language-Project.git

In [24]:
!pip install ffmpeg

In [25]:
# !python /content/Crework-Sign-Language-Project/vid.py -f /content/Crework-Sign-Language-Project/videos/svideo2.mp4 -s 3 -v h264

In [26]:
import os
os.chdir('/content/Crework-Sign-Language-Project')
from vid import split_by_seconds,get_video_length
vid_length = get_video_length('/content/Crework-Sign-Language-Project/videos/vid.mp4')
split_by_seconds(filename='/content/Crework-Sign-Language-Project/videos/vid.mp4',split_length=3,vcodec='h264',video_length=vid_length)

Video length in seconds: 12


In [27]:
mode = 'rgb'
num_classes = 2000
save_model = './checkpoints/'

 
## Change to where the videos are located
root = {'word':'videos'}

In [28]:
train_split = 'preprocess/nslt_2000.json'

weights = '/content/Crework-Sign-Language-Project/checkpoints/nslt_2000_065846_0.447803.pt'

In [29]:
import shutil
import os
shutil.move('/content/Crework-Sign-Language-Project/videos/vid.mp4','/content/vid.mp4')
os.remove('/content/Crework-Sign-Language-Project/videos/t')

In [30]:
import cv2 
import os
import random
import json
complete_dict = dict()
os.chdir('/content/Crework-Sign-Language-Project/videos')
for video in os.listdir():
  if video == ".ipynb_checkpoints":
    continue
  id = video.split('.')[0]
  capture = cv2.VideoCapture(video)
  frameNr = 0 
  while (True):
      success, frame = capture.read() 
      if success:
       frameNr = frameNr+1
      else:
        break 
  
  dictionary = {
      str(id):{
          "subset":"test",
          "action":[random.randint(0,2000),1,frameNr]
      }
  }
  complete_dict.update(dictionary)
json_object = json.dumps(complete_dict, indent = 4)
with open("/content/sample.json", "w") as outfile:
    outfile.write(json_object)
capture.release()

In [31]:
from torchvision import transforms
os.chdir('/content/Crework-Sign-Language-Project')
import videotransforms
import numpy as np
from datasets.nslt_dataset import NSLT as Dataset

test_transforms = transforms.Compose([videotransforms.CenterCrop(224)])
dataset = Dataset("/content/sample.json", 'test', root, mode, test_transforms)

Video #1 / 4 - videos/vid-3-of-4.mp4
Video #2 / 4 - videos/vid-1-of-4.mp4
Video #3 / 4 - videos/vid-4-of-4.mp4
Video #4 / 4 - videos/vid-2-of-4.mp4
Skipped videos:  0
Total number of videos in the dataset:  4


In [32]:
datasets = {'test': dataset}
import pickle
with open('datasets.pkl', 'wb') as f:
    pickle.dump(datasets, f)

In [33]:
import pickle
with open('datasets.pkl', 'rb') as f:
   datasets = pickle.load(f)

In [34]:
from test_i3d import *

In [35]:
run(mode=mode, root=root, train_split="/content/sample.json", weights=weights, datasets=datasets, num_classes=num_classes)

1162
1 / 4 vid-3-of-4 0.0 0.0 0.0
1163
2 / 4 vid-1-of-4 0.0 0.0 0.0
1222
3 / 4 vid-4-of-4 0.0 0.0 0.0
147
4 / 4 vid-2-of-4 0.0 0.0 0.0
top-k average per class acc: 0.0, 0.0, 0.0


In [36]:
with open('/content/Crework-Sign-Language-Project/predictions.txt') as f:
  pred = f.readlines()
f.close()

In [37]:
words = [i.split('\n')[0] for i in pred ]
words

['1162', '1163', '1222', '147']

In [38]:
with open('/content/Crework-Sign-Language-Project/preprocess/wlasl_class_list.txt') as f:
      labels = f.readlines()

In [39]:
map = {}
for element in labels:
  first = element.split('\t')
  second = first[1].split('\n')[0]
  map[first[0]] = second

In [40]:
text_words = [map[num] for num in words]
text_words

['manager', 'maximum', 'shampoo', 'cold']